In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import scipy as sp
import datetime as dt
import os
import utils as u

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [14]:
FILE_PATH = os.path.abspath(os.curdir+"\\..\\data-processing\\data\\funds.pkl")
funds_df_raw = pd.read_pickle(FILE_PATH)

Os dados provenientes do scraper projetado estão bem brutos e tem que ser processados. Ainda por cima, o dataset apresenta muitos valores faltantes que, ou devem ser preenchidos, ou eliminados.

In [15]:
funds_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Ticker                         213 non-null    object        
 1   Nome                           213 non-null    object        
 2   Administrador                  213 non-null    object        
 3   Descrição                      190 non-null    object        
 4   Data de Constituição do Fundo  87 non-null     datetime64[ns]
 5   Cotas Emitidas                 212 non-null    float64       
 6   Tipo de Gestão                 204 non-null    object        
 7   Público Alvo                   209 non-null    object        
 8   Mandato                        204 non-null    object        
 9   Segmento                       204 non-null    object        
 10  Prazo de Duração               213 non-null    object        
 11  Taxa de Administraç

Foi criado o seguinte pipeline (cujo desenvolvimento pode ser observado no arquivo "data-preparation-test.ipynb") que consistem em uma série de processamentos como descrito a seguir:

1) A coluna de descrição contém o texto bruto coletado site, sendo assim, são removidos quaisquer cabeçalhos e em seguida as pontuações.

2) Em seguida são inseridos os valores "0,2% a.a" nas linhas em que a taxa de administração está faltante. Como descrito no relatório, essa é uma taxa "padrão" do mercado, já que não havia a opção de inserir a "média do texto".

3) O mesmo processo da coluna "Descrição" é executado para "Taxa de Adminsitração"

4) Muitos dos fundos não tinham a sua data de constituição, sendo assim, quando disponível, foi usada a data da primeira cotação disponível (o que reflete o IPO do fundo).

5) No caso da liquidez diária foi implementada a mediana, já que a média poderia estar distorcida. Há uma grande disparidade de valores negociados no mesmo dia entre todos os fundos.

6) Para as colunas "Dividendos Históricos", "Cotações Históricas", "Valor Patrimonial Histórica" e "Vacância Histórica" foi usada a mesma técnica: destrinchar a série temporal nos valores mensais até M-11 e obter parâmetros estatísticos como a média, os valores máximo e mínimo, o desvio padrão, a curtose e a assimetria. Em alguns, o total acumulado e em outros a máxima variação.

7) Na coluna dos Ativos do fundo, foi separado por unidade federativa a área disponível.

8) Por último foram removidas colunas irrelevantes após a adição de features e as linhas que continham valores vazios; codificados os textos e variáveis categóricas e convertidas as colunas numéricas em números com ponto flutuante.

In [16]:
cat_columns = ['Administrador', 'Tipo de Gestão', 'Público Alvo', 'Mandato', 'Segmento', 'Prazo de Duração']
text_columns = ['Descrição', 'Taxa de Administração']

# stores the categories information after vectorization
cat_manager = u.CategoryManager()

clustering_pipeline = Pipeline([   
    #1
    ('clean-description-headers', u.CleanHeaders(col=3)),
    ('clean-description-punct', u.CleanPunct(col=3)),
    #2
    ('input-const-admin-tax', u.FillColumn(col=11, method='const', const='0,2% a.a.')),
    #3
    ('clean-admin-tax-headers', u.CleanHeaders(col=11)),
    ('clean-admin-tax-punct', u.CleanPunct(col=11)),
    #4
    ('input-foundation-date', u.InputDate(col=4, ref_col=16)),
    #5
    ('input-mean-daily-liquidity', u.FillColumn(col=14, method='median')),
    #6
    ('process-dividends', u.ProcessDividends(col=17)),
    ('process-prices', u.ProcessPrices(col=16)),
    ('process-equity', u.ProcessEquity(col=19)),
    ('process-vacancy', u.ProcessVacancy(col=20)),
    #7
    ('process-assets', u.ProcessAssets(col=13)),
    #8
    ('drop-columns', u.DropColumns(cols=["Taxa de Performance", "Ativos Atuais", "Cotações Históricas",
                                         "Dividendos Históricos", "Dividend Yield Histórico", "Nome",
                                         "Valor Patrimonial Histórico", "Vacância Histórica", 'Descrição',
                                         'Taxa de Administração'])),
    ('drop-rows', u.DropRows(rows='all')),
#     ('standard-scaler', u.StdScaler(exclude_col=['Div. Acum. Últ. Trimestre', 'Data de Constituição do Fundo'])),
    ('cat-encoding', u.OneHotEncoder2(col=cat_columns, category_manager=cat_manager)),
#     ('text-encoding', u.CountVectorizer2(col=text_columns)),
    ('convert-available2float', u.Convert2Float())
]) 

In [17]:
funds_df = clustering_pipeline.fit_transform(funds_df_raw)

C:\Users\lucas\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3584: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\lucas\anaconda3\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Running Convert2Float
Unable to convert column 'Ticker' to float
Unable to convert column 'Data de Constituição do Fundo' to float


In [18]:
funds_df.head(n=3)

Ticker Data de Constituição do Fundo  Cotas Emitidas  Liquidez Diária  \
0  ABCP11                    2007-01-08       1000000.0           3152.0   
1  ALZR11                    2018-01-04        997042.0          11570.0   
2  ATSA11                    2010-12-13        100000.0            121.0   

   Patrimônio Líquido  Div. M-0  Div. M-1  Div. M-2  Div. M-3  Div. M-4  ...  \
0        1.000000e+09  0.000000  0.000000  0.490000  0.830000  0.490000  ...   
1        2.700000e+08  0.596226  0.586388  0.586496  0.594587  0.567433  ...   
2        1.900000e+08  0.250000  0.250000  0.310000  0.250000  0.250000  ...   

   Hotel  Híbrido  Lajes Corporativas  Logística  Outros  Residencial  \
0    0.0      0.0                 0.0        0.0     0.0          0.0   
1    0.0      0.0                 0.0        1.0     0.0          0.0   
2    0.0      0.0                 0.0        0.0     0.0          0.0   

   Shoppings  Títulos e Val. Mob.  Determinado  Indeterminado  
0        1.0                  0.0          0.0            1.0  
1        0.0                  0.0          0.0            1.0  
2        1.0                  0.0          0.0            1.0  

[3 rows x 154 columns]

In [19]:
funds_df.describe()

Cotas Emitidas  Liquidez Diária  Patrimônio Líquido    Div. M-0  \
count    1.210000e+02       121.000000        1.210000e+02  121.000000   
mean     2.802031e+06     13243.487603        5.404375e+08    1.650376   
std      8.506707e+06     52587.021604        7.506904e+08    4.246405   
min      1.000000e+02         1.000000        4.000000e+04    0.000000   
25%      1.950000e+05       212.000000        1.300000e+08    0.350000   
50%      7.500000e+05      2074.000000        2.400000e+08    0.590000   
75%      1.798000e+06     11530.000000        6.300000e+08    0.890000   
max      6.903350e+07    567091.000000        4.100000e+09   30.090745   

         Div. M-1    Div. M-2    Div. M-3    Div. M-4    Div. M-5    Div. M-6  \
count  121.000000  121.000000  121.000000  121.000000  121.000000  121.000000   
mean     1.574019    1.729948    1.759749    2.250348    2.069111    1.826682   
std      4.465990    4.278422    4.374880    5.671280    4.641415    4.284573   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.260000    0.400000    0.500000    0.520000    0.530000    0.450000   
50%      0.534130    0.637606    0.700000    0.700000    0.700000    0.648770   
75%      0.760000    0.890000    1.000000    1.000000    1.216060    1.000000   
max     29.748247   28.425929   29.044033   40.500000   29.159220   28.252900   

       ...       Hotel     Híbrido  Lajes Corporativas   Logística  \
count  ...  121.000000  121.000000          121.000000  121.000000   
mean   ...    0.008264    0.090909            0.231405    0.099174   
std    ...    0.090909    0.288675            0.423484    0.300138   
min    ...    0.000000    0.000000            0.000000    0.000000   
25%    ...    0.000000    0.000000            0.000000    0.000000   
50%    ...    0.000000    0.000000            0.000000    0.000000   
75%    ...    0.000000    0.000000            0.000000    0.000000   
max    ...    1.000000    1.000000            1.000000    1.000000   

           Outros  Residencial   Shoppings  Títulos e Val. Mob.  Determinado  \
count  121.000000   121.000000  121.000000           121.000000   121.000000   
mean     0.123967     0.024793    0.140496             0.256198     0.066116   
std      0.330914     0.156141    0.348946             0.438348     0.249517   
min      0.000000     0.000000    0.000000             0.000000     0.000000   
25%      0.000000     0.000000    0.000000             0.000000     0.000000   
50%      0.000000     0.000000    0.000000             0.000000     0.000000   
75%      0.000000     0.000000    0.000000             1.000000     0.000000   
max      1.000000     1.000000    1.000000             1.000000     1.000000   

       Indeterminado  
count     121.000000  
mean        0.933884  
std         0.249517  
min         0.000000  
25%         1.000000  
50%         1.000000  
75%         1.000000  
max         1.000000  

[8 rows x 152 columns]

In [20]:
funds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Columns: 154 entries, Ticker to Indeterminado
dtypes: datetime64[ns](1), float64(152), object(1)
memory usage: 145.7+ KB


In [21]:
X, info = funds_df.drop(columns=['Ticker']), list(funds_df['Ticker'])
km = KModes(n_clusters=10, init='Huang', n_init=50, verbose=True)
clusters = km.fit_predict(X, categorical=[i for i in range(106, len(list(funds_df.columns)))])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 3, cost: 7894.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 15, cost: 7897.0
Run 2, iteration: 2/100, moves: 5, cost: 7890.0
Run 2, iteration: 3/100, moves: 0, cost: 7890.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 3, cost: 7901.0
Run 3, iteration: 2/100, moves: 1, cost: 7900.0
Run 3, iteration: 3/100, moves: 0, cost: 7900.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 13, cost: 7892.0
Run 4, iteration: 2/100, moves: 3, cost: 7887.0
Run 4, iteration: 3/100, moves: 3, cost: 7877.0
Run 4, iteration: 4/100, moves: 0, cost: 7877.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 7, cost: 7881.0
Run 5, iteration: 2/10

Run 42, iteration: 2/100, moves: 1, cost: 7901.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 43, iteration: 1/100, moves: 13, cost: 7897.0
Run 43, iteration: 2/100, moves: 0, cost: 7897.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 44, iteration: 1/100, moves: 14, cost: 7882.0
Run 44, iteration: 2/100, moves: 1, cost: 7881.0
Run 44, iteration: 3/100, moves: 1, cost: 7881.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 45, iteration: 1/100, moves: 2, cost: 7914.0
Run 45, iteration: 2/100, moves: 0, cost: 7914.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 46, iteration: 1/100, moves: 26, cost: 7924.0
Run 46, iteration: 2/100, moves: 1, cost: 7924.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 47, iteration: 1/100, moves: 6, cost: 7893.0
Run 47, iteration: 2/100, moves: 0, cost: 7893.0
Init: ini

In [23]:
names_clustered = [[] for _ in range(len(list(np.unique(clusters))))]

for ix, cluster in enumerate(clusters):
    names_clustered[cluster].append(info[ix])
    
for ix, cluster in enumerate(names_clustered):
    print(f'Cluster { ix }')
    for ticker in cluster:
        segmento = funds_df_raw[funds_df_raw["Ticker"] == ticker]["Segmento"].item()
        mandato = funds_df_raw[funds_df_raw["Ticker"] == ticker]["Mandato"].item()   
        gestao = funds_df_raw[funds_df_raw["Ticker"] == ticker]["Tipo de Gestão"].item()  
        div = funds_df[funds_df["Ticker"] == ticker]["Div. Acum. Últ. Trimestre"].item()
        
        print(f'{ ticker }: { segmento } | { mandato } | { gestao } | R${ round(div,2) }')
    
    print("\n")

Cluster 0
ABCP11: Shoppings | Renda | Passiva | R$0.49
ATSA11: Shoppings | Desenvolvimento para Renda | Ativa | R$0.81
BBRC11: Outros | Desenvolvimento para Renda | Passiva | R$2.78
BNFS11: Outros | Desenvolvimento para Renda | Ativa | R$2.82
CTXT11: Lajes Corporativas | Desenvolvimento para Renda | Ativa | R$1.14
CXCE11B: Outros | Desenvolvimento para Renda | Passiva | R$61.36
CXTL11: Logística | Renda | Passiva | R$0.39
DMAC11: Residencial | Desenvolvimento para Venda | Passiva | R$10.12
EURO11: Logística | Híbrido | Passiva | R$1.95
FIGS11: Shoppings | Desenvolvimento para Venda | Passiva | R$0.2
FMOF11: Lajes Corporativas | Renda | Passiva | R$0.77
FVPQ11: Shoppings | Renda | Passiva | R$2.61
GRLV11: Logística | Desenvolvimento para Renda | Passiva | R$1.92
HUSC11: Outros | Desenvolvimento para Renda | Passiva | R$0.59
KNRE11: Residencial | Desenvolvimento para Venda | Ativa | R$0.0
NVHO11: Hospital | Desenvolvimento para Renda | Ativa | R$0.18
RBCB11: Outros | Renda | Ativa | R$0.

In [24]:
# kmeans = KMeans(n_clusters=10, random_state=RANDOM_SEED);
# kmeans.fit(X_train)

In [33]:
# drop nas colunas que tem interferência com a label
X_train = funds_df[0:100].drop(columns=["Div. Acum. Últ. Trimestre", "Div. M-0", "Div. M-1", "Div. M-2",
                                        "Preços Média M-0", "Preços Média M-1", "Preços Média M-2",
                                        "Val. Patr. M-0", "Val. Patr. M-1", "Val. Patr. M-2", "Vacância M-0",
                                        "Vacância M-1", "Vacância M-2", "Ticker", "Data de Constituição do Fundo"])

X_test = funds_df[100:120].drop(columns=["Div. Acum. Últ. Trimestre", "Div. M-0", "Div. M-1", "Div. M-2",
                                        "Preços Média M-0", "Preços Média M-1", "Preços Média M-2",
                                        "Val. Patr. M-0", "Val. Patr. M-1", "Val. Patr. M-2", "Vacância M-0",
                                        "Vacância M-1", "Vacância M-2", "Ticker", "Data de Constituição do Fundo"])

y_train = funds_df[["Div. Acum. Últ. Trimestre"]][0:100]

y_test = funds_df[["Div. Acum. Últ. Trimestre"]][100:120]

In [34]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [48]:
y_pred = lin_reg.predict(X_test).ravel()
y_real = np.array(y_test).ravel()

for i, y in enumerate(y_pred):
    print(f'P: { y } | R: { y_real[i] } | D: { y-y_real[i] }')

P: 5.407826068741343 | R: 9.1 | D: -3.6921739312586563
P: 3.049566820084492 | R: 1.63 | D: 1.419566820084492
P: 1.6339434848644594 | R: 0.7404815170000001 | D: 0.8934619678644593
P: 7.8320252437679345 | R: 7.35 | D: 0.4820252437679349
P: 0.77094005072474 | R: 2.8699999999999997 | D: -2.0990599492752597
P: 1.5467874446257683 | R: 1.621427675 | D: -0.07464023037423173
P: 2.400524995971446 | R: 1.3399999999999999 | D: 1.0605249959714462
P: 0.178627814275508 | R: 2.4 | D: -2.221372185724492
P: 2.5977231655823596 | R: 1.58 | D: 1.0177231655823595
P: -6.882671374386676 | R: 0.8 | D: -7.6826713743866755
P: -1.2006654956684217 | R: 1.30110161 | D: -2.5017671056684216
P: 2.183924608975606 | R: 1.08 | D: 1.103924608975606
P: 33.14287795081089 | R: 2.1799999999999997 | D: 30.962877950810892
P: -0.5455800597430862 | R: 1.35 | D: -1.8955800597430863
P: 8.321344191164936 | R: 2.1645000000000003 | D: 6.156844191164936
P: -0.8536387507948833 | R: 0.5 | D: -1.3536387507948833
P: -0.8513487965025437 | R